In [32]:
Base.compilecache(Base.PkgId(ProtoSyn))

┌ Info: Recompiling stale cache file /Users/ssantos/.julia/compiled/v1.1/ProtoSyn/8FAXX.ji for ProtoSyn [c9758760-7c0d-11e9-0ffc-fb9355b7d293]
└ @ Base loading.jl:1184
┌ Warning: Package ProtoSyn does not have LinearAlgebra in its dependencies:
│ - If you have ProtoSyn checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ProtoSyn
└ Loading LinearAlgebra into ProtoSyn from project dependency, future warnings for ProtoSyn are suppressed.


"/Users/ssantos/.julia/compiled/v1.1/ProtoSyn/8FAXX.ji"

In [33]:
using ProtoSyn

# Creating a residue

A residue is a named list of atoms `ProtoSyn.Atom[]` and connectivity graph `ProtoSyn.ConnectGraph`or `ProtoSyn.ConnectGraphByName`.

Example:

In [51]:
residue = ProtoSyn.Residue(
    "BKB",
    [
        Atom( 1,    "H",    "H",    3.909/10,    0.724/10,    0.000),
        Atom( 2,    "N",    "N",    3.326/10,    1.548/10,    0.000),
        Atom( 3,    "C",   "CA",    3.970/10,    2.846/10,    0.000),
        Atom( 4,    "C",    "C",    5.486/10,    2.705/10,    0.000),
        Atom( 5,    "O",    "O",    6.009/10,    1.593/10,    0.000)
    ],
    ConnectGraph(
        1  => [    2 ],
        2  => [    1,    3 ],
        3  => [    2,    4 ],
        4  => [    3,    5 ],
        5  => [    4 ]
    )
)

5-atom Residue:
   name = BKB
  atoms = 5-element array
  bonds = 5-element dict

In [52]:
residue.atoms

5-element Array{ProtoSyn._Atom{Residue},1}:
 Atom:
     id = 1
   name = H
 symbol = H
 parent = BKB 
 Atom:
     id = 2
   name = N
 symbol = N
 parent = BKB 
 Atom:
     id = 3
   name = CA
 symbol = C
 parent = BKB
 Atom:
     id = 4
   name = C
 symbol = C
 parent = BKB 
 Atom:
     id = 5
   name = O
 symbol = O
 parent = BKB 

# Residue Library

A residue library is a dictionary of resname=>residue (`Dict{String,Residue}`)

Example:

In [53]:
lib = ProtoSyn.ResidueLib()
lib[residue.name] = residue
# or 
# ProtoSyn.ResidueLib(residue.name => residue)

5-atom Residue:
   name = BKB
  atoms = 5-element array
  bonds = 5-element dict

# Linked Residue

A residue is a template for constructing molecules. An instance of
residue within a molecule happens through instantiation of a `ProtoSyn.LinkedResidue`,
having a `ProtoSyn.Residue` as its source. Hence, a residue can be used multiple times
(remember: it is a simple template).
A LinkedResidue has an id, a source, and can have multiple links (`Link`) to
other (multiple) linked residues.

Example:
two LinkedResidue(s) of type BKB (not linked to each other)

In [54]:
lr1 = ProtoSyn.LinkedResidue(id=1, source=residue)
lr2 = ProtoSyn.LinkedResidue(id=2, source=residue)

5-atom LinkedResidue:
      id = 2
  source = BKB
  offset = 0
   links = 0-element array

# Molecule


A molecule is a list of linked residues (`LinkedResidue[]`) and a
list of links (`Link[]`) linking those residues. Nevertheless, links 
between linked residues are not mandatory (may be changed in the future).

Whenever bulding, or changing, a molecule, the `ProtoSyn.update!` function must be called.
This ensures internal consistency of the `Molecule` data structures.

### Example:

In [55]:
mol = ProtoSyn.Molecule(residues=[lr1, lr2])
ProtoSyn.update!(mol)
println("molecule is coherent? ", mol.coherent)

molecule is coherent? true


### list connectivity graph:

In [56]:
mol.bonds

Dict{Int64,Array{Int64,1}} with 10 entries:
  7  => [6, 8]
  4  => [3, 5]
  9  => [8, 10]
  10 => [9]
  2  => [1, 3]
  3  => [4, 2]
  5  => [4]
  8  => [9, 7]
  6  => [7]
  1  => [2]

### list residues:

In [57]:
mol.residues

2-element Array{LinkedResidue,1}:
 5-atom LinkedResidue:
      id = 1
  source = BKB
  offset = 0
   links = 0-element array
 5-atom LinkedResidue:
      id = 2
  source = BKB
  offset = 5
   links = 0-element array

### list links:

since no links were provided so far, both linked residues within the molecule are not actually linked!

In [58]:
mol.links

0-element Array{ProtoSyn._Link{LinkedResidue},1}

# Links

A link `ProtoSyn.Link` is a structure that takes to linked residues (`LinkedResidue`) and a connectivity graph (`ConnectGraph`)
that indicates which atoms from linked residue 1 connect to which atoms of linked residue 2. This connectivity graph always uses
local atom indices.

### Example

Link `lr1` and `lr2` through atom atom 4 of `lr1` (`C`) and atom 2 of `lr2` (`N`):

**NOTE**: a single atom from l-residue 1 can connect to multiple atoms of l-residue 2! 

In [59]:
# instantiate link
link = ProtoSyn.Link(lr1, lr2, ProtoSyn.ConnectGraph(4 => [2]))
# add link to molecule
push!(mol, link)
# add link to bothe linked residues
push!(lr1, link)
push!(lr2, link)
link

1-bond Link:
  residue1 = BKB.1
  residue2 = BKB.2
  bonds:
   4 => 2

In [60]:
println("molecule is coherent: ", mol.coherent)

molecule is coherent: false


In [61]:
ProtoSyn.update!(mol)

ATOM      1    H BKB    1  H -> [2]
ATOM      2    N BKB    1  N -> [1, 3]
ATOM      3   CA BKB    1  C -> [4, 2]
ATOM      4    C BKB    1  C -> [3, 5, 7]
ATOM      5    O BKB    1  O -> [4]
ATOM      6    H BKB    2  H -> [7]
ATOM      7    N BKB    2  N -> [6, 8, 4]
ATOM      8   CA BKB    2  C -> [9, 7]
ATOM      9    C BKB    2  C -> [8, 10]
ATOM     10    O BKB    2  O -> [9]


In [62]:
mol.links

1-element Array{ProtoSyn._Link{LinkedResidue},1}:
 1-bond Link:
  residue1 = BKB.1
  residue2 = BKB.2
  bonds:
   4 => 2

Alternatively, one can use the `peptidebond` rule from the `ProtoSyn.Peptides` module to
straightforwardly bind those two l-residues:

In [63]:
# link = ProtoSyn.bind(ProtoSyn.Peptides.peptidebond, lr1, lr2)
# if link !== nothing
#     ProtoSyn.update!(mol)
# end

# Build a polypeptide out of the box

Instead of manually building a molecule as shown above, one can use the utility method in `ProtoSyn.Peptides`
to automatically build a peptide chain for a given sequence, using the user-provided residue library.


### Example

build a molecule with 5 `BKB` residues, using the above generated lib:

In [64]:
sequence = repeat(["BKB"], 5)
mol, state = ProtoSyn.Peptides.build(sequence, lib);

Alternativelly, `ProtoSyn.Peptides.one_2_three` provides a mapping between 1-letter codes and 3-letter residue names.

The `BKB` (backbone) aminoacid is coded as `?`. Hence, the molecule can be constructed from a singe string and a user-provided library:

In [65]:
mol, state = ProtoSyn.Peptides.build("?????", lib);
#   or simply
# mol, state = ProtoSyn.Peptides.build("?"^5, lib);

In [66]:
# display((mol,state)) -> Tuple{Molecule,State}
mol,state

(ATOM      1    H BKB    1  H -> [2]
ATOM      2    N BKB    1  N -> [1, 3]
ATOM      3   CA BKB    1  C -> [4, 2]
ATOM      4    C BKB    1  C -> [3, 5, 7]
ATOM      5    O BKB    1  O -> [4]
ATOM      6    H BKB    2  H -> [7]
ATOM      7    N BKB    2  N -> [6, 8, 4]
ATOM      8   CA BKB    2  C -> [9, 7]
ATOM      9    C BKB    2  C -> [8, 10, 12]
ATOM     10    O BKB    2  O -> [9]
ATOM     11    H BKB    3  H -> [12]
ATOM     12    N BKB    3  N -> [11, 13, 9]
ATOM     13   CA BKB    3  C -> [14, 12]
ATOM     14    C BKB    3  C -> [13, 15, 17]
ATOM     15    O BKB    3  O -> [14]
ATOM     16    H BKB    4  H -> [17]
ATOM     17    N BKB    4  N -> [16, 18, 14]
ATOM     18   CA BKB    4  C -> [19, 17]
ATOM     19    C BKB    4  C -> [18, 20, 22]
ATOM     20    O BKB    4  O -> [19]
ATOM     21    H BKB    5  H -> [22]
ATOM     22    N BKB    5  N -> [21, 23, 19]
ATOM     23   CA BKB    5  C -> [24, 22]
ATOM     24    C BKB    5  C -> [23, 25]
ATOM     25    O BKB    5  O -> [24]
, State{Float64}(25, [0.0124844 0.128164 -0.0; 0.0 0.028 -0.0; … ; 1.71304 0.028 -0.0; 1.70663 0.150718 -0.0], nothing, nothing, ProtoSyn.Energy{Float64}(NaN, Dict{Symbol,Float64}())))

# Crankshaft and Dihedral Identification

The `ProtoSyn.Peptides` module offers a par of methods for identifying rotatable crankshafts and dihedrals in peptides.

### Example:

In [67]:
crankshafts = ProtoSyn.Peptides.findcrankshafts(mol)

10-element Array{AxisRotatableBlock,1}:
 AxisRotatableBlock(8, 13, 9)  
 AxisRotatableBlock(8, 18, 9)  
 AxisRotatableBlock(8, 23, 9)  
 AxisRotatableBlock(3, 8, 4)   
 AxisRotatableBlock(3, 13, 4)  
 AxisRotatableBlock(3, 18, 4)  
 AxisRotatableBlock(3, 23, 4)  
 AxisRotatableBlock(13, 18, 14)
 AxisRotatableBlock(13, 23, 14)
 AxisRotatableBlock(18, 23, 19)

In [68]:
# backbone and sidechain dihedrals are identified by default.
# to select which one wishes to get, simply do:
#   ProtoSyn.Peptides.finddihedrals(mol, sidechain=<true|false>, backbone=<true|false>)
dihedrals = ProtoSyn.Peptides.finddihedrals(mol)

12-element Array{Dihedral,1}:
 Dihedral(2, 3, 4, 7, 4, psi::Type = 2)       
 Dihedral(3, 4, 7, 8, 7, omega::Type = 3)     
 Dihedral(4, 7, 8, 9, 8, phi::Type = 1)       
 Dihedral(7, 8, 9, 12, 9, psi::Type = 2)      
 Dihedral(8, 9, 12, 13, 12, omega::Type = 3)  
 Dihedral(9, 12, 13, 14, 13, phi::Type = 1)   
 Dihedral(12, 13, 14, 17, 14, psi::Type = 2)  
 Dihedral(13, 14, 17, 18, 17, omega::Type = 3)
 Dihedral(14, 17, 18, 19, 18, phi::Type = 1)  
 Dihedral(17, 18, 19, 22, 19, psi::Type = 2)  
 Dihedral(18, 19, 22, 23, 22, omega::Type = 3)
 Dihedral(19, 22, 23, 24, 23, phi::Type = 1)  

# State

A `ProtoSyn.State` is a data structure that stores the current state of a `ProtoSyn.Molecule`.
It stores coordinates, velocities, forces, (...). In the examples above, a State is returned by the `build` method.

In [69]:
# copy state
state1 = ProtoSyn.State(state);

### Example - rotate dihedral 4 by 90º

In [71]:
dihd = dihedrals[4]
ProtoSyn.rotate!(state1, mol, dihd, deg2rad(90))
display(dihd)
(mol,state1)

Dihedral(7, 8, 9, 12, 9, psi::Type = 2)

(ATOM      1    H BKB    1  H -> [2]
ATOM      2    N BKB    1  N -> [1, 3]
ATOM      3   CA BKB    1  C -> [4, 2]
ATOM      4    C BKB    1  C -> [3, 5, 7]
ATOM      5    O BKB    1  O -> [4]
ATOM      6    H BKB    2  H -> [7]
ATOM      7    N BKB    2  N -> [6, 8, 4]
ATOM      8   CA BKB    2  C -> [9, 7]
ATOM      9    C BKB    2  C -> [8, 10, 12]
ATOM     10    O BKB    2  O -> [9]
ATOM     11    H BKB    3  H -> [12]
ATOM     12    N BKB    3  N -> [11, 13, 9]
ATOM     13   CA BKB    3  C -> [14, 12]
ATOM     14    C BKB    3  C -> [13, 15, 17]
ATOM     15    O BKB    3  O -> [14]
ATOM     16    H BKB    4  H -> [17]
ATOM     17    N BKB    4  N -> [16, 18, 14]
ATOM     18   CA BKB    4  C -> [19, 17]
ATOM     19    C BKB    4  C -> [18, 20, 22]
ATOM     20    O BKB    4  O -> [19]
ATOM     21    H BKB    5  H -> [22]
ATOM     22    N BKB    5  N -> [21, 23, 19]
ATOM     23   CA BKB    5  C -> [24, 22]
ATOM     24    C BKB    5  C -> [23, 25]
ATOM     25    O BKB    5  O -> [24]
, State{Float64}(25, [0.0124844 0.128164 -0.0; 0.0 0.028 -0.0; … ; 0.991068 -1.06322 -1.6446e-16; 0.875619 -1.10531 -2.36427e-16], nothing, nothing, ProtoSyn.Energy{Float64}(NaN, Dict{Symbol,Float64}())))

In [72]:
### Example - rotate dihedral 8 by 90º

In [73]:
dihd = dihedrals[8]
ProtoSyn.rotate!(state1, mol, dihd, deg2rad(90))
display(dihd)
(mol,state1)

Dihedral(13, 14, 17, 18, 17, omega::Type = 3)

(ATOM      1    H BKB    1  H -> [2]
ATOM      2    N BKB    1  N -> [1, 3]
ATOM      3   CA BKB    1  C -> [4, 2]
ATOM      4    C BKB    1  C -> [3, 5, 7]
ATOM      5    O BKB    1  O -> [4]
ATOM      6    H BKB    2  H -> [7]
ATOM      7    N BKB    2  N -> [6, 8, 4]
ATOM      8   CA BKB    2  C -> [9, 7]
ATOM      9    C BKB    2  C -> [8, 10, 12]
ATOM     10    O BKB    2  O -> [9]
ATOM     11    H BKB    3  H -> [12]
ATOM     12    N BKB    3  N -> [11, 13, 9]
ATOM     13   CA BKB    3  C -> [14, 12]
ATOM     14    C BKB    3  C -> [13, 15, 17]
ATOM     15    O BKB    3  O -> [14]
ATOM     16    H BKB    4  H -> [17]
ATOM     17    N BKB    4  N -> [16, 18, 14]
ATOM     18   CA BKB    4  C -> [19, 17]
ATOM     19    C BKB    4  C -> [18, 20, 22]
ATOM     20    O BKB    4  O -> [19]
ATOM     21    H BKB    5  H -> [22]
ATOM     22    N BKB    5  N -> [21, 23, 19]
ATOM     23   CA BKB    5  C -> [24, 22]
ATOM     24    C BKB    5  C -> [23, 25]
ATOM     25    O BKB    5  O -> [24]
, State{Float64}(25, [0.0124844 0.128164 -0.0; 0.0 0.028 -0.0; … ; 1.19726 -0.836744 -0.306275; 1.15509 -0.798353 -0.415127], nothing, nothing, ProtoSyn.Energy{Float64}(NaN, Dict{Symbol,Float64}())))

In [74]:
### Example - rotate crankshaft 2 by 90º

In [76]:
crank = crankshafts[2]
ProtoSyn.rotate!(state1, mol, crank, deg2rad(90))
display(crank)
(mol,state1)

AxisRotatableBlock(8, 18, 9)

(ATOM      1    H BKB    1  H -> [2]
ATOM      2    N BKB    1  N -> [1, 3]
ATOM      3   CA BKB    1  C -> [4, 2]
ATOM      4    C BKB    1  C -> [3, 5, 7]
ATOM      5    O BKB    1  O -> [4]
ATOM      6    H BKB    2  H -> [7]
ATOM      7    N BKB    2  N -> [6, 8, 4]
ATOM      8   CA BKB    2  C -> [9, 7]
ATOM      9    C BKB    2  C -> [8, 10, 12]
ATOM     10    O BKB    2  O -> [9]
ATOM     11    H BKB    3  H -> [12]
ATOM     12    N BKB    3  N -> [11, 13, 9]
ATOM     13   CA BKB    3  C -> [14, 12]
ATOM     14    C BKB    3  C -> [13, 15, 17]
ATOM     15    O BKB    3  O -> [14]
ATOM     16    H BKB    4  H -> [17]
ATOM     17    N BKB    4  N -> [16, 18, 14]
ATOM     18   CA BKB    4  C -> [19, 17]
ATOM     19    C BKB    4  C -> [18, 20, 22]
ATOM     20    O BKB    4  O -> [19]
ATOM     21    H BKB    5  H -> [22]
ATOM     22    N BKB    5  N -> [21, 23, 19]
ATOM     23   CA BKB    5  C -> [24, 22]
ATOM     24    C BKB    5  C -> [23, 25]
ATOM     25    O BKB    5  O -> [24]
, State{Float64}(25, [0.0124844 0.128164 -0.0; 0.0 0.028 -0.0; … ; 1.19726 -0.836744 -0.306275; 1.15509 -0.798353 -0.415127], nothing, nothing, ProtoSyn.Energy{Float64}(NaN, Dict{Symbol,Float64}())))

# IO

Printing a molecule + state as a pdb/xyz file

In [80]:
write(stdout, (mol, state))

MethodError: MethodError: no method matching write(::IJulia.IJuliaStdio{Base.PipeEndpoint}, ::Tuple{Molecule,State{Float64}})
Closest candidates are:
  write(::IO, ::Any) at io.jl:498
  write(::IO, ::Any, !Matched::Any...) at io.jl:500
  write(::IO, !Matched::Complex) at complex.jl:217
  ...

In [81]:
write(stdout, mol, state, ProtoSyn.XYZ)

25
title
H          0.125   1.282  -0.000
N          0.000   0.280  -0.000
C          1.181  -0.560  -0.000
C          2.450   0.280  -0.000
O          2.386   1.507  -0.000
H          3.795  -1.282   0.000
N          3.670  -0.280   0.000
C          4.851   0.560   0.000
C          6.120  -0.280   0.000
O          6.056  -1.507   0.000
H          7.465   1.282  -0.000
N          7.340   0.280  -0.000
C          8.521  -0.560  -0.000
C          9.790   0.280  -0.000
O          9.726   1.507  -0.000
H         11.135  -1.282   0.000
N         11.010  -0.280   0.000
C         12.191   0.560   0.000
C         13.460  -0.280   0.000
O         13.396  -1.507   0.000
H         14.805   1.282  -0.000
N         14.680   0.280  -0.000
C         15.861  -0.560  -0.000
C         17.130   0.280  -0.000
O         17.066   1.507  -0.000


In [82]:
phis   = filter(d -> d.type==ProtoSyn.Peptides.DihedralTypes.phi, dihedrals)
psis   = filter(d -> d.type==ProtoSyn.Peptides.DihedralTypes.psi, dihedrals)
omegas = filter(d -> d.type==ProtoSyn.Peptides.DihedralTypes.omega, dihedrals)

for (i,phi) in enumerate(phis)
    println("ϕ$(i) =", rad2deg(ProtoSyn.measure(phi, state1)))
end

ϕ1 =-158.9068939794619
ϕ2 =-180.0
ϕ3 =-130.791136833356
ϕ4 =179.9999999999999


In [83]:
mol.links

4-element Array{ProtoSyn._Link{LinkedResidue},1}:
 1-bond Link:
  residue1 = BKB.1
  residue2 = BKB.2
  bonds:
   4 => 2
 1-bond Link:
  residue1 = BKB.2
  residue2 = BKB.3
  bonds:
   4 => 2
 1-bond Link:
  residue1 = BKB.3
  residue2 = BKB.4
  bonds:
   4 => 2
 1-bond Link:
  residue1 = BKB.4
  residue2 = BKB.5
  bonds:
   4 => 2